In [17]:
import pandas as pd
import numpy as np
from keras.models import model_from_json

In [18]:
COLUMNS = ['x-axis', 'y-axis', 'z-axis']
LABELS = ['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking']
MODEL_PATH = "../models/model.json"
WEIGHTS_PATH = "../models/model.h5"

In [19]:
def process_and_load_WISDM_file_to_memory(file_path):
    dataset = pd.read_csv(file_path, header=None, names=LABELS)
    dataset = dataset.dropna(inplace=False)
    dataset = dataset.round({'x-axis': 5, 'y-axis': 5, 'z-axis': 5})
    return dataset


def create_data_segmentation(dataset, window, increment):
    segmented_dataset = []

    for i in range(0, len(dataset) - window, increment):
        x_axis = dataset['x-axis'].values[i: i + window]
        y_axis = dataset['y-axis'].values[i: i + window]
        z_axis = dataset['z-axis'].values[i: i + window]
        segmented_dataset.append([x_axis, y_axis, z_axis])

    segmented_dataset = np.asarray(segmented_dataset, dtype=np.float32)
    transposed_segments = segmented_dataset.reshape(-1, window, 3)
    return transposed_segments


def load_model(model_path, weights_path):
    json_file = open(model_path, 'r')
    loaded_model = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model)
    loaded_model.load_weights(weights_path)
    return loaded_model


def process_raw_data(path):
    df = pd.read_csv(path, header=None, names=COLUMNS)
    return df


def classify_activity(model, processed_data):
    keras_prediction = np.argmax(model.predict(processed_data), axis=1)
    return LABELS[keras_prediction[0]]

In [20]:
paths = ['raw_standing.txt','raw_sitting.txt','raw_walking.txt',
         'raw_downstairs.txt','raw_upstairs.txt','raw_jogging.txt']

for path in paths:
    model = load_model(MODEL_PATH, WEIGHTS_PATH)
    raw_data = process_raw_data("../data/iPhone_raw/"+path)
    segmented_data = create_data_segmentation(raw_data, 200, 20)
    output = classify_activity(model, segmented_data)
    print(path)
    print(output+"\n")

raw_standing.txt
Standing

raw_sitting.txt
Sitting

raw_walking.txt
Standing

raw_downstairs.txt
Downstairs

raw_upstairs.txt
Upstairs

raw_jogging.txt
Jogging

